<a href="https://colab.research.google.com/github/nachodele/UFV---Visualizacion/blob/main/Practicas/practica1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 1 || Histórico de ventas
## 4ºB
## Ingeniería Matemática || Visualización de datos
0. Cargamos datos e importamos librerías

In [3]:
import pandas as pd
import plotly.express as px

# Cargar datos
data = pd.read_csv('datos_ejercicio_ventas.csv')

1. Como se distribuyen las ventas realizadas en:  
• Cada país






In [19]:
# Boxplot
fig_pais = px.box(data, x='COUNTRY', y='AMOUNT', title="Distribución de Ventas por País")
fig_pais.show()

• Cada mes y año

In [41]:
# Media del AMOUNT para obtener ventas totales por mes cada año
data_med = data.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()

# Bar Plot acumulado
fig_mes = px.bar(data_med, x='MONTH', y='AMOUNT', color='YEAR', title="Distribución de Ventas por Mes y Año")
fig_mes.update_layout(
    yaxis_title="SUM OF AMOUNT"
)
fig_mes.show()


• Cada marca

In [10]:
# Violin Plot
fig_marca = px.violin(data, x='SUBBRAND', y='AMOUNT', box=True, points="all", title="Distribución de Ventas por Marca")
fig_marca.show()

2. Cual es la tendencia y estacionalidad de:  
• Todas las ventas del país con menos ventas

In [37]:
# Identificar país con menos ventas
pais_menos_ventas = data.groupby('COUNTRY')['AMOUNT'].sum().idxmin()
data_pais_menos = data[data['COUNTRY'] == pais_menos_ventas]

# Media del AMOUNT para obtener ventas totales por mes cada año
data_pais_menos_med = data_pais_menos.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()

# Mostramos la tendencia con Area Plot
fig_tendencia_pais = px.area(data_pais_menos_med, x='MONTH', y='AMOUNT', color='YEAR', title=f"Tendencia de Ventas en {pais_menos_ventas}")
fig_tendencia_pais.show()

• La marca con más ventas

In [36]:
# Identificar marca con más ventas
marca_mas_ventas = data.groupby('SUBBRAND')['AMOUNT'].sum().idxmax()
data_marca_mas = data[data['SUBBRAND'] == marca_mas_ventas]

# Media del AMOUNT para obtener ventas totales por mes cada año
data_marca_mas_med = data_marca_mas.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()

# Evolución de ventas con Area Plot
fig_tendencia_marca = px.area(data_marca_mas_med, x='MONTH', y='AMOUNT', color='YEAR', title=f"Tendencia de Ventas para la Marca {marca_mas_ventas}")
fig_tendencia_marca.show()

3. Cuales son las predicciones hechas en España y como de buenas son.

In [46]:
# Filtrar datos para España y el escenario de predicción
data_espana = data[data['COUNTRY'] == 'Spain']

# Media de cada mes por año para cada tipo de predicción
data_espana_med = data_espana.groupby(['FORECAST', 'MONTH'])['AMOUNT'].mean().reset_index()

# Comparación de predicciones y valores reales (Vertical Bar Chart)
fig_prediccion = px.bar(data_espana_med, x='MONTH', y='AMOUNT', color='FORECAST', title="Comparación de Predicciones y Valores Reales en España")
fig_prediccion.show()